In [ ]:
import tensorflow_constrained_optimization as tfco
import os
import sys
import tempfile
import urllib3

import math
import random
import numpy as np
import pandas as pd
import warnings
from six.moves import xrange
import tensorflow.compat.v1 as tf
import tensorflow_constrained_optimization as tfco
import matplotlib.pyplot as plt

tf.disable_eager_execution()

warnings.filterwarnings('ignore')
%matplotlib inline
from tensorflow import keras
from tensorflow.keras import Model

# import tensorflow_datasets as tfds
# tfds.disable_progress_bar()

from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn import svm, neighbors, tree
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier

from fairlearn.reductions import ExponentiatedGradient
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

import shap
shap.initjs()

from IPython.display import display, Markdown, Latex, HTML
import statistics

from fairlearn.reductions import GridSearch, DemographicParity, ErrorRate, TruePositiveRateParity
import numpy as np
from datetime import datetime

# Get Data

In [ ]:
CATEGORICAL_COLUMNS = [
    'workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country'
]
CONTINUOUS_COLUMNS = [
    'age', 'capital_gain', 'capital_loss', 'hours_per_week', 'education_num'
]
COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]
LABEL_COLUMN = 'label'

PROTECTED_COLUMNS = [
    'gender_Female'
]

def get_data():
    train_df_raw = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=COLUMNS, skipinitialspace=True)
    test_df_raw = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", names=COLUMNS, skipinitialspace=True, skiprows=1)

    train_df_raw[LABEL_COLUMN] = (train_df_raw['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
    test_df_raw[LABEL_COLUMN] = (test_df_raw['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
    # Preprocessing Features
    pd.options.mode.chained_assignment = None  # default='warn'

    # Functions for preprocessing categorical and continuous columns.
    def binarize_categorical_columns(input_train_df, input_test_df, categorical_columns=[]):

        def fix_columns(input_train_df, input_test_df):
            test_df_missing_cols = set(input_train_df.columns) - set(input_test_df.columns)
            for c in test_df_missing_cols:
                input_test_df[c] = 0
                train_df_missing_cols = set(input_test_df.columns) - set(input_train_df.columns)
            for c in train_df_missing_cols:
                input_train_df[c] = 0
                input_train_df = input_train_df[input_test_df.columns]
            return input_train_df, input_test_df

        # Binarize categorical columns.
        binarized_train_df = pd.get_dummies(input_train_df, columns=categorical_columns)
        binarized_test_df = pd.get_dummies(input_test_df, columns=categorical_columns)
        # Make sure the train and test dataframes have the same binarized columns.
        fixed_train_df, fixed_test_df = fix_columns(binarized_train_df, binarized_test_df)
        return fixed_train_df, fixed_test_df

    def bucketize_continuous_column(input_train_df,
                                  input_test_df,
                                  continuous_column_name,
                                  num_quantiles=None,
                                  bins=None):
        assert (num_quantiles is None or bins is None)
        if num_quantiles is not None:
            train_quantized, bins_quantized = pd.qcut(
              input_train_df[continuous_column_name],
              num_quantiles,
              retbins=True,
              labels=False)
            input_train_df[continuous_column_name] = pd.cut(
              input_train_df[continuous_column_name], bins_quantized, labels=False)
            input_test_df[continuous_column_name] = pd.cut(
              input_test_df[continuous_column_name], bins_quantized, labels=False)
        elif bins is not None:
            input_train_df[continuous_column_name] = pd.cut(
              input_train_df[continuous_column_name], bins, labels=False)
            input_test_df[continuous_column_name] = pd.cut(
              input_test_df[continuous_column_name], bins, labels=False)

    # Filter out all columns except the ones specified.
    train_df = train_df_raw[CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS + [LABEL_COLUMN]]
    test_df = test_df_raw[CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS + [LABEL_COLUMN]]
    
    # Bucketize continuous columns.
    bucketize_continuous_column(train_df, test_df, 'age', num_quantiles=4)
    bucketize_continuous_column(train_df, test_df, 'capital_gain', bins=[-1, 1, 4000, 10000, 100000])
    bucketize_continuous_column(train_df, test_df, 'capital_loss', bins=[-1, 1, 1800, 1950, 4500])
    bucketize_continuous_column(train_df, test_df, 'hours_per_week', bins=[0, 39, 41, 50, 100])
    bucketize_continuous_column(train_df, test_df, 'education_num', bins=[0, 8, 9, 11, 16])
    train_df, test_df = binarize_categorical_columns(train_df, test_df, categorical_columns=CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS)
    feature_names = list(train_df.keys())
    feature_names.remove(LABEL_COLUMN)
    num_features = len(feature_names)
    
    return train_df, test_df, feature_names
train_df, test_df, FEATURE_NAMES = get_data()

In [ ]:
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

class BuildFunction:
    def __init__(self, D):
        self.D = D
        
    def __call__(self):
        model = Sequential()
        model.add(Dense(100, input_dim=D, activation='relu'))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[
                                                                        tf.keras.metrics.Accuracy(), 
                                                                        tf.keras.metrics.Recall(),
                                                                        tf.keras.metrics.TruePositives(),
                                                                        tf.keras.metrics.Precision()
                                                                        ])
        return model
        
def create_model_creator(D):
    def create_keras_model():
        model = Sequential()
        model.add(Dense(100, input_dim=D, activation='relu'))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[
                                                                        tf.keras.metrics.Accuracy(), 
                                                                        tf.keras.metrics.Recall(),
                                                                        tf.keras.metrics.TruePositives(),
                                                                        tf.keras.metrics.Precision()
                                                                        ])
        return model
    return create_keras_model

class LogRegTF(Model):
    def __init__(self, D):
        super(LogRegTF, self).__init__()
        self.W = tf.Variable(tf.random.normal(shape=[D, 1]))
        self.b = tf.Variable(tf.random.normal(shape=[1, 1]))

    def call(self, x):
        print((tf.matmul(x, self.W) + self.b).shape)
        return tf.matmul(x, self.W) + self.b

class BuildFunctionLogReg:
    def __init__(self, D, opt='adam'):
        self.D = D
        self.opt = opt
        
    def __call__(self):
        model = Sequential()
        model.add(Dense(1, input_dim=D, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer=self.opt, metrics=[
                                                                        tf.keras.metrics.Accuracy(), 
                                                                        tf.keras.metrics.Recall(),
                                                                        tf.keras.metrics.TruePositives(),
                                                                        tf.keras.metrics.Precision()
                                                                        ])
        return model

def fetch_sk_model(name, input_dim, optimizer='adam'):
    if name=='mlp':
        return MLPClassifier(hidden_layer_sizes=(100, 20), verbose=True, max_iter=10000, solver=optimizer, learning_rate_init=3e-3)
    elif name=='nn-keras':
        return tf.keras.wrappers.scikit_learn.KerasClassifier(
            build_fn=BuildFunction(input_dim), verbose=False)
    elif name=='logistic':
        return LogisticRegression(solver='liblinear', fit_intercept=True)
    elif name=='logistic-keras':
        return tf.keras.wrappers.scikit_learn.KerasClassifier(
            build_fn=BuildFunctionLogReg(input_dim, opt=optimizer), verbose=False)
    elif name=='logistic-sgd':
        return SGDClassifier(loss='log')


In [ ]:
D = len(FEATURE_NAMES)
unmit_nn = fetch_sk_model('logistic', D)
unmit_nn.fit(train_df[FEATURE_NAMES].to_numpy(), train_df[LABEL_COLUMN].to_numpy())

In [ ]:
X_train_all, y_train_all, A_train_all = train_df[FEATURE_NAMES], train_df[LABEL_COLUMN], train_df[PROTECTED_COLUMNS]


In [ ]:
def run_expgrad(model, X_tr, A_tr, y_tr, constraint=TruePositiveRateParity, 
                X_te=None, A_te=None, y_te=None, eps=0.01, nu=1e-6,
                max_iter=50,
                eta0=2.0):
    expgrad = ExponentiatedGradient(
        model,
        constraints=constraint(), eps=eps, nu=nu,
        eta0=eta0,
        max_iter=max_iter
    )
    if getattr(model, "summary", None) is not None:
        model.summary()
    else:
        print(f"Model used: {model!r}")
    a=datetime.now()
    expgrad.fit(X_tr, y_tr,
                           sensitive_features=A_tr)
    b=datetime.now()
    time_expgrad_all = (b-a).seconds
    
    def Qexp_all(X): return expgrad._pmf_predict(X)[:, 1]
    
    # violation of log res
    disparity_moment = constraint()
    disparity_moment.load_data(X_tr, y_tr,
                               sensitive_features=A_tr)
    violation_expgrad_all = disparity_moment.gamma(Qexp_all).max()

    # error of log res
    error = ErrorRate()
    error.load_data(X_tr, y_tr,
                    sensitive_features=A_tr)
    error_expgrad_all = error.gamma(Qexp_all)[0]

    print('[TRAINING] Exponentiated gradient : Time: {} seconds; Violation: {}; Error: {}'.format(
        time_expgrad_all, violation_expgrad_all, error_expgrad_all))
    
    if X_te is not None and y_te is not None and A_te is not None:
        # violation of log res
        disparity_moment = constraint()
        disparity_moment.load_data(X_te, y_te,
                                   sensitive_features=A_te)
        te_violation_expgrad_all = disparity_moment.gamma(Qexp_all).max()

        # error of log res
        error = ErrorRate()
        error.load_data(X_te, y_te,
                        sensitive_features=A_te)
        te_error_expgrad_all = error.gamma(Qexp_all)[0]

        print('[TEST] Exponentiated gradient : Violation: {}; Error: {}'.format(te_violation_expgrad_all, 
                                                                           te_error_expgrad_all))
    return expgrad

def run_unmitigated(model, X_tr, A_tr, y_tr, constraint=TruePositiveRateParity, name='None'):
    model.fit(X_tr, y_tr)
    def Qexp_all(X): return model.predict_proba(X)[:, 1]
    
    # violation of log res
    disparity_moment = constraint()
    disparity_moment.load_data(X_tr, y_tr,
                               sensitive_features=A_tr)
    violation_expgrad_all = disparity_moment.gamma(Qexp_all).max()

    # error of log res
    error = ErrorRate()
    error.load_data(X_tr, y_tr,
                    sensitive_features=A_tr)
    error_expgrad_all = error.gamma(Qexp_all)[0]

    print('[{}] Exponentiated gradient : Violation: {}; Error: {}'.format(
        name, violation_expgrad_all, error_expgrad_all))
    return model

# run_expgrad(fetch_sk_model('logistic', D), X_train_all, A_train_all, y_train_all)

## Unmitigated run

In [ ]:
# run experiments
print('\n Logistic Regression with sklearn')
expgrad_logistic = run_expgrad(fetch_sk_model('logistic-sgd', D), X_train_all, A_train_all, y_train_all)
run_unmitigated(fetch_sk_model('logistic-sgd', D), X_train_all, A_train_all, y_train_all, name='Unmitigated LogReg sklearn')

display(Markdown('***'))
print('Logistic Regression with Keras')
expgrad_logistic_keras = run_expgrad(fetch_sk_model('logistic-keras', D, optimizer='sgd'), X_train_all, A_train_all, y_train_all)
run_unmitigated(fetch_sk_model('logistic-keras', D, optimizer='sgd'), X_train_all, A_train_all, y_train_all, name='Unmitigated LogReg keras')

display(Markdown('***'))
print('Neural Network with Keras')
expgrad_nn_keras = run_expgrad(fetch_sk_model('nn-keras', D, optimizer='sgd'), X_train_all, A_train_all, y_train_all)
run_unmitigated(fetch_sk_model('nn-keras', D, optimizer='sgd'), X_train_all, A_train_all, y_train_all, name='Unimitigated NN keras')

display(Markdown('***'))
print('\n Neural Network with sklearn')
expgrad_nn_keras = run_expgrad(fetch_sk_model('mlp', D), X_train_all, A_train_all, y_train_all)
run_unmitigated(fetch_sk_model('mlp', D, optimizer='sgd'), X_train_all, A_train_all, y_train_all, name='Unimitigated NN sklearn')